In [10]:
# ===============================
# CÉLULA 0 — CHECK DE GPU
# ===============================

import torch, sys

print("Torch:", torch.__version__)
print("CUDA disponível:", torch.cuda.is_available())

if not torch.cuda.is_available():
    raise RuntimeError(
        "❌ GPU NÃO ATIVA\n"
        "Ative em: Ambiente de execução → Alterar tipo → GPU\n"
        "Depois REINICIE o runtime."
    )

print("GPU detectada:", torch.cuda.get_device_name(0))
print("✅ GPU OK — pode continuar")


Torch: 2.9.0+cu126
CUDA disponível: True
GPU detectada: Tesla T4
✅ GPU OK — pode continuar


In [11]:
# ===============================
# CÉLULA 1 — HUGGING FACE LOGIN
# ===============================

import os
from huggingface_hub import login

if "HUGGINGFACE_TOKEN" in os.environ:
    login(token=os.environ["HUGGINGFACE_TOKEN"])
    print("✅ Hugging Face autenticado com token")
else:
    print("⚠️ HUGGINGFACE_TOKEN não encontrado")
    print("➡️ Acesso público será usado (ok para Qwen 7B)")


⚠️ HUGGINGFACE_TOKEN não encontrado
➡️ Acesso público será usado (ok para Qwen 7B)


In [12]:
# ===============================
# CÉLULA 2 — SETUP
# ===============================

import os, sys, subprocess

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

def pip_install(args):
    subprocess.check_call([sys.executable, "-m", "pip"] + args.split())

def pip_uninstall(pkg):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", pkg])
    except:
        pass

# NÃO reinstala torch (Colab já vem com CUDA correto)
pip_install("install -q -U transformers accelerate huggingface_hub")

pip_uninstall("bitsandbytes")
pip_install("install -q -U bitsandbytes")

print("✅ Ambiente configurado")


✅ Ambiente configurado


In [13]:
# ===============================
# CÉLULA 3 — MODELO QWEN 2.5 7B
# ===============================

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

MODEL_ID = "Qwen/Qwen2.5-7B-Instruct"

# Tokenizer (leve, não ocupa VRAM)
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    use_fast=True,
    trust_remote_code=True
)

# Quantização 4-bit otimizada para T4
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# 🚫 NUNCA usar device_map="auto" no Colab Free
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map={"": 0},          # força TUDO na GPU
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,      # evita estouro de RAM
    trust_remote_code=True,
)

model.eval()

print("✅ Qwen2.5-7B carregado com sucesso")
print("VRAM usada (GB):", round(torch.cuda.memory_allocated() / 1024**3, 2))


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Qwen2.5-7B carregado com sucesso
VRAM usada (GB): 8.33


In [14]:
# ===============================
# CÉLULA 4 — INFERÊNCIA SEGURA
# ===============================

def ask_qwen(
    prompt,
    max_new_tokens=700,
    temperature=0.2,
    top_p=0.95,
    max_prompt_tokens=6000,  # CONTROLE REAL
    show_usage=False
):
    messages = [
        {"role": "system", "content": "Você é um especialista em Engenharia de Software e DevOps."},
        {"role": "user", "content": prompt},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        truncation=True,
        max_length=max_prompt_tokens,
        return_tensors="pt"
    ).to(model.device)

    if show_usage:
        print("Prompt tokens:", input_ids.shape[-1])

    with torch.inference_mode():
        output = model.generate(
            input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id,
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    return decoded.split(messages[-1]["content"])[-1].strip()


In [15]:
# ===============================
# CÉLULA 5 — TESTE
# ===============================

resp = ask_qwen(
    "Explique em 3 linhas o que é GitFlow.",
    show_usage=True
)

print(resp)


Prompt tokens: 38
linha
GitFlow é um padrão de gerenciamento de versões para o Git que estende o modelo de desenvolvimento branching do Git, fornecendo um fluxo de trabalho mais estruturado para projetos complexos com várias linhas de desenvolvimento simultâneas.


In [16]:
# ===============================
# CÉLULA 6 — COLETA GIT
# ===============================

import subprocess, os
from collections import Counter
from IPython.display import Markdown, display

def git(cmd, cwd):
    return subprocess.check_output(cmd, cwd=cwd, shell=True).decode().strip()

repo_url = "https://github.com/unclecode/crawl4ai.git"
repo_dir = "/content/repo"

if not os.path.exists(repo_dir):
    subprocess.check_call(["git", "clone", repo_url, repo_dir])

branches = git("git branch -r | grep -v HEAD", repo_dir).splitlines()
branches = [b.replace("origin/", "").strip() for b in branches]

tags = git("git tag --list", repo_dir).splitlines()

bucket = Counter(b.split('/')[0] if '/' in b else 'root' for b in branches)

display(Markdown(f"- **Branches:** {len(branches)}"))
display(Markdown(f"- **Tags:** {len(tags)}"))
display(Markdown(f"```{bucket}```"))
# ===============================
# CÉLULA 6 — COLETA GIT
# ===============================

import subprocess, os
from collections import Counter
from IPython.display import Markdown, display

def git(cmd, cwd):
    return subprocess.check_output(cmd, cwd=cwd, shell=True).decode().strip()

repo_url = "https://github.com/unclecode/crawl4ai.git"
repo_dir = "/content/repo"

if not os.path.exists(repo_dir):
    subprocess.check_call(["git", "clone", repo_url, repo_dir])

branches = git("git branch -r | grep -v HEAD", repo_dir).splitlines()
branches = [b.replace("origin/", "").strip() for b in branches]

tags = git("git tag --list", repo_dir).splitlines()

bucket = Counter(b.split('/')[0] if '/' in b else 'root' for b in branches)

display(Markdown(f"- **Branches:** {len(branches)}"))
display(Markdown(f"- **Tags:** {len(tags)}"))
display(Markdown(f"```{bucket}```"))


- **Branches:** 148

- **Tags:** 40

```Counter({'root': 77, 'fix': 27, 'feature': 14, 'release': 9, 'codex': 6, 'docker': 3, 'claude': 2, 'feat': 2, 'unclecode': 2, 'bug': 1, 'bugfix': 1, 'coderabbitai': 1, 'copilot': 1, 'devin': 1, 'patch': 1})```

- **Branches:** 148

- **Tags:** 40

```Counter({'root': 77, 'fix': 27, 'feature': 14, 'release': 9, 'codex': 6, 'docker': 3, 'claude': 2, 'feat': 2, 'unclecode': 2, 'bug': 1, 'bugfix': 1, 'coderabbitai': 1, 'copilot': 1, 'devin': 1, 'patch': 1})```

In [17]:
# ===============================
# CÉLULA 7 — ANÁLISE
# ===============================

prompt = f"""
Analise o modelo de branching com base nos dados reais.

Branches totais: {len(branches)}
Distribuição:
{bucket}

Amostra:
{sorted(branches)[:40]}

Classifique:
A) GitFlow
B) GitHub Flow
C) Trunk-Based Development

Justifique com evidências reais.
"""

resp = ask_qwen(prompt)
display(Markdown("## 🌿 Branching Model"))
display(Markdown(resp))


## 🌿 Branching Model

.imgur.com

Baseado na análise dos dados fornecidos, podemos classificar o modelo de branching como sendo mais próximo do **GitHub Flow**. Aqui estão as evidências que justificam essa classificação:

### Evidências para GitHub Flow:

1. **Branches de Desenvolvimento Principal (root):**
   - A maioria das branches (`root`) indica que a maioria do trabalho de desenvolvimento está sendo feito diretamente no branch principal. Isso é característico do GitHub Flow, onde todas as features e correções são feitas em branches separadas e depois mescladas no branch principal (`master` ou `main`).

2. **Branches de Correção de Erros (fix):**
   - Há uma quantidade significativa de branches de correção de erros (`fix`). Isso é comum no GitHub Flow, onde pequenas correções de bugs são feitas em branches separadas e depois mescladas no branch principal.

3. **Branches de Features (feature):**
   - Existem várias branches de features (`feature`). No GitHub Flow, cada feature é desenvolvida em sua própria branch e depois mesclada no branch principal.

4. **Lack of Release Branches:**
   - Não há evidências claras de branches de lançamento (`release`), que são comuns no GitFlow. Em GitFlow, cada release tem seu próprio branch separado.

5. **Branches de Codificação e Outras:**
   - Branches como `codex`, `docker`, `claude`, `copilot`, etc., indicam que há um fluxo de trabalho onde várias features e correções são desenvolvidas em branches separadas e depois mescladas no branch principal.

### Evidências Contra GitFlow e Trunk-Based Development:

1. **GitFlow:**
   - **Release Branches:** Não há evidências de branches de lançamento (`release`), que são essenciais no GitFlow.
   - **Stable Branches:** Em GitFlow, há um branch `develop` que serve como um branch de integração contínua, mas aqui não há evidências disso.

2. **Trunk-Based Development:**
   - **Single Main Branch:** Trunk-Based Development envolve o uso de apenas um branch principal (`trunk`), onde todas as features e correções são mescladas. Aqui, há várias branches separadas (`root`, `fix`, `feature`, etc.), o que não é típico do Trunk-Based Development.

### Conclusão:

O modelo de branching apresentado é mais consistente com o **GitHub Flow**, pois:
- Usa um branch principal (`root`) para o desenvolvimento.
- Usa branches separadas para features e correções de erros.
- Falta a presença de branches de lançamento (`release`), que são características do GitFlow.
- Não segue a abordagem de um único branch principal (`trunk`), que é típica do Trunk-Based Development.

Portanto, a classificação mais adequada para este modelo de branching é **GitHub Flow**.

In [9]:
# ===============================
# CÉLULA 8 — RELEASES
# ===============================

prompt = f"""
Analise a estratégia de releases com base nas tags abaixo.

Total de tags: {len(tags)}
Amostra de tags:
{tags[-20:]}

Classifique:
- Rapid Release
- Release Train
- LTS + Current
- Ad-hoc
"""

print(ask_qwen(prompt))


:response Para analisar a estratégia de releases com base nas tags fornecidas, vamos considerar algumas características comuns a cada tipo de estratégia:

1. **Rapid Release**: Caracteriza-se por lançamentos frequentes e pequenos incrementos.
2. **Release Train**: Envolve lançamentos regulares em intervalos definidos, mas geralmente maiores que os rapid releases.
3. **LTS + Current**: Inclui lançamentos de Long Term Support (LTS) e uma versão atual.
4. **Ad-hoc**: Lançamentos não programados ou de acordo com necessidades específicas.

Agora, vamos classificar as tags com base nessas características:

### Rapid Release
- **Característica**: Versões menores e mais frequentes.
- **Tags**: 
  - `v0.3.3`, `v0.3.6`, `v0.3.745`, `v0.3.746`, `v0.4.24`, `v0.4.243`, `v0.5.0.post1`, `v0.6.3`, `v0.7.0`, `v0.7.1`, `v0.7.2`, `v0.7.3`, `v0.7.4`, `v0.7.5`, `v0.7.6`, `v0.7.7`, `v0.7.8`
  
  **Justificativa**: Essas tags indicam versões menores e mais frequentes, sugerindo uma estratégia de rapid releas